### Recall@k
What percentage of true value are retrieved from actual true values.
$$recall=\frac{truePositives}{truePositives+falseNegatives}

In [3]:
# recall@k function
def recall(actual, predicted, k):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = round(len(act_set & pred_set)) / float(len(act_set))
    return result

In [4]:
actual = ["2", "4", "5", "7"]
predicted = ["1", "2", "3", "4", "5", "6", "7", "8"]

for k in range(1, 9):
    print(f'Recall@{k} = {recall(actual, predicted, k)}')

Recall@1 = 0.0
Recall@2 = 0.25
Recall@3 = 0.25
Recall@4 = 0.5
Recall@5 = 0.75
Recall@6 = 0.75
Recall@7 = 1.0
Recall@8 = 1.0


### MRR (Mean reciprocal rank)
Considers order of the result.
$$MRR=\frac{1}{Q} \sum_{q=1}^Q \frac{1}{rank_q}$$
$rank_q$ = rank of first relevant item in the rank q position

ideal scenario = 1

In [5]:
# relevant result for query #1, #2 and #3
actual_relevant = [
    [2, 4, 5, 7],
    [1, 4, 5, 7],
    [5, 8]
]

In [6]:
# number of queries
Q = len(actual_relevant)

# calculate reciprocal of the first actual relevant
reciprocal = 0
for i in range(Q):
    first_result = actual_relevant[i][0]
    reciprocal = reciprocal + (1 / first_result)
    print(f"query #{i+1} = 1/{first_result} = {reciprocal}")


# calculate mrr
mrr = 1 / Q * reciprocal

# generate results
print("MRR = ", round(mrr, 2))

query #1 = 1/2 = 0.5
query #2 = 1/1 = 1.5
query #3 = 1/5 = 1.7
MRR =  0.57


### MAP@K (Mean average precision)
Mean of average precision
$$MAP@K=\frac{1}{Q} \sum_{q=1}^Q AP@K_q$$
where,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;AP = Average precision

$$AP@K=\frac{\sum_{k=1}^K (Precision@k*rel_k)}{\#\ of\ relevant\ results}$$
where,<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$rel_k$ = relevant or not at position k (0, 1). It ensures to calculate precision for relevant item.

and,
$$Precision@k=\frac{truePositives}{truePositives+falsePositives}$$


In [8]:
# initialize variables
actual = [
    [2, 4, 5, 7],
    [1, 4, 5, 7],
    [5, 8]
]

Q = len(actual)
predicted = [1, 2, 3, 4, 5, 6, 7, 8]
K = 8
ap = []

# loop through and calculate AP for each query q
for q in range(Q):
    ap_num = 0
    # loop through k values
    for x in range(K):
        # calculate Precision@K
        act_set = set(actual[q])
        pred_set = set(predicted[:x+1])
        precision_at_K = len(act_set & pred_set) / (x+1)

        # calculate rel_k values
        if predicted[x] in actual[q]:
            rel_k = 1
        else:
            rel_k = 0
        
        # calculate numerator value for ap
        ap_num += precision_at_K * rel_k
    
    # now we calculate AP value as the average of ap_num
    ap_q = ap_num/len(actual[q])
    print(f'AP@{K}_{q+1} = {round(ap_q, 2)}')
    ap.append(ap_q)

# now we take the mean of all ap values to get MAP
map_at_K = sum(ap) / Q

# generate result
print(f"MAP@{K} = {round(map_at_K, 2)}")

AP@8_1 = 0.54
AP@8_2 = 0.67
AP@8_3 = 0.23
MAP@8 = 0.48


### NDCG@K (Normalized Discounted Cumulative Gain)

$$CG@K=\sum_{k=1}^K rel_k$$
Here,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$rel_k$ = value from 0 to 4 by relevancy.

Also, CG is not in order.

$$DCG@K=\sum_{k=1}^K \frac{rel_k}{log_2(1+k)}$$
DCG is in order.

In [11]:
from math import log2

# initialize variables
relevance = [0, 4, 1, 3, 4, 1, 3, 2]
K = 8

dcg = 0
# loop through each item and calculate DCG
for k in range(1, K+1):
    rel_k = relevance[k-1]

    # calculate DCG
    dcg += rel_k / log2(1+k)
    print(f"DCG@{k} = {round(dcg, 2)}")

DCG@1 = 0.0
DCG@2 = 2.52
DCG@3 = 3.02
DCG@4 = 4.32
DCG@5 = 5.86
DCG@6 = 6.22
DCG@7 = 7.22
DCG@8 = 7.85


DCG values depends on relevancy score. If we give value other than 0-4, then it is completely different. So, it is hard to interpret.

$$NDCG@K=\frac{DCG@K}{IDCG@K}$$
Here,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;IDCG@K (Ideal DCG) = rank item in order by relevancy and calculate DCG.

In [12]:
# Sort item in relevance from most relevant to less
ideal_relevance = sorted(relevance, reverse=True)

print(ideal_relevance)

idcg = 0

# as before loop through each item and calculate ideal DCG
for k in range(1, K+1):
    rel_k = ideal_relevance[k-1]

    # calculate DCG
    idcg += rel_k / log2(1+k)
    print(f"IDCG@{k} = {round(idcg, 2)}")

[4, 4, 3, 3, 2, 1, 1, 0]
IDCG@1 = 4.0
IDCG@2 = 6.52
IDCG@3 = 8.02
IDCG@4 = 9.32
IDCG@5 = 10.09
IDCG@6 = 10.45
IDCG@7 = 10.78
IDCG@8 = 10.78


In [13]:
# Calculating NDCG@K
dcg = 0
idcg = 0

for k in range(1, K+1):
    # calculate rel_k values
    rel_k = relevance[k-1]
    ideal_rel_k = ideal_relevance[k-1]

    # calculate DCG and IDCG
    dcg += rel_k / log2(1+k)
    idcg += ideal_rel_k / log2(1+k)

    # calculate NDCG
    ndcg = dcg / idcg

    print(f"NDCG@{k} = {round(ndcg, 2)}")

NDCG@1 = 0.0
NDCG@2 = 0.39
NDCG@3 = 0.38
NDCG@4 = 0.46
NDCG@5 = 0.58
NDCG@6 = 0.6
NDCG@7 = 0.67
NDCG@8 = 0.73
